In [1]:
# Basic UI for Strobe-Enhanced Microscopy Stage

# ===== IMPORTS =====
import base64
import serial
import time
import atexit
import threading
from datetime import datetime
from IPython.display import display, HTML, clear_output, Image
import ipywidgets as widgets
import requests

# ===== HARDWARE COMMUNICATION =====
PI_IP = '192.168.137.3'
#PI_IP = '0.0.0.0'
BASE_URL = f'http://{PI_IP}:5000/api'

# ===== MAIN UI LAYOUT =====
def create_ui():
    
    # Create tab container
    tab = widgets.Tab()
    
    # Create output containers
    status_output = widgets.Output()
    
    # ===== MICROSCOPY TAB =====
    stream_btn = widgets.ToggleButton(description='▶ Start Camera', button_style='success')
    capture_btn = widgets.Button(description='📸 Capture Image')
    save_container = widgets.Output()
    enable_btn = widgets.ToggleButton(description='⏻ Enable Strobe', button_style='success')
    period = widgets.FloatSlider(min=1, max=10000, step=1, value=50, description='Period (µs):')
    width = widgets.FloatSlider(min=0.1, max=1000, step=0.1, value=0.1, description='Width (µs):')
    hold_btn = widgets.ToggleButton(description='🔆 Hold Mode')
    stream_container = widgets.Output()
    captured_container = widgets.Output()
  
    def update_strobe(change=None):
        try:
            data = {
                'enable': enable_btn.value,
                'period_ns': int(period.value * 1000),  # Convert to nanoseconds
                'width_ns': int(width.value * 1000),    # Convert to nanoseconds
                'hold': hold_btn.value
            }
            response = requests.post(f'{BASE_URL}/strobe/settings', json=data)
            #response.raise_for_status()
            #with status_output:
            #    print("✅ Strobe settings updated")
        except Exception as e:
            with status_output:
                print(f"❌ Error updating strobe: {e}")

    def toggle_stream(change):
        if change['new']:
            stream_btn.description = '⏹ Stop Camera'
            stream_btn.button_style = 'danger'
            with stream_container:
                clear_output()
                display(HTML(f"""
                    <img src="{BASE_URL}/camera/stream" 
                         style="max-width: 400px; max-height: 300px; border: 2px solid #4CAF50; border-radius: 4px;">
                """))
        else:
            stream_btn.description = '▶ Start Camera'
            stream_btn.button_style = 'success'
            with stream_container:
                clear_output()
                display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">Stream Stopped</div>'))

    def on_strobe_toggle(change):
        if change['name'] == 'value':
            if change['new']:  # Button is being enabled
                enable_btn.description = '⏹ Disable Strobe'
                enable_btn.button_style = 'danger'
            else:  # Button is being disabled
                enable_btn.description = '⏻ Enable Strobe'
                enable_btn.button_style = 'success'
        update_strobe()

    def capture_image(btn):
        try:
            response = requests.get(f'{BASE_URL}/camera/capture')
            response.raise_for_status()
            if response.status_code == 200:
                with captured_container:
                    clear_output()
                    display(Image(data=response.content, width=400))
                with save_container:
                    clear_output()
                    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                    display(HTML(f"""
                        <a href="data:image/jpeg;base64,{base64.b64encode(response.content).decode('utf-8')}"
                           download="capture_{timestamp}.jpg"
                           style="padding: 6px 12px; background-color: #4CAF50; color: white; 
                                  text-decoration: none; border-radius: 4px; display: inline-block;">
                            💾 Save Image
                        </a>
                    """))
                with status_output:
                    print("✅ Image captured! Click 'Save Image' to download.")
        except Exception as e:
            with status_output:
                print(f"❌ Error capturing image: {e}")         
    
    # ===== LAYOUT =====
    camera_controls = widgets.HBox([stream_btn, capture_btn, save_container])
    strobe_controls = widgets.VBox([
        widgets.HBox([enable_btn, hold_btn]),
        period,
        width])
    image_display = widgets.HBox([
        widgets.VBox([widgets.Label('Live Stream'), stream_container]),
        widgets.VBox([widgets.Label('Captured Image'), captured_container])
    ])
    
    microscopy_tab = widgets.VBox([
        widgets.HTML("<h3>Camera Controls</h3>"),
        camera_controls,
        widgets.HTML("<h3>Strobe Controls</h3>"),
        strobe_controls,
        widgets.HTML("<hr>"),
        image_display
    ])
    
    # Set up tabs
    tab.children = [microscopy_tab]
    tab.titles = ['Microscope']
    
    # Display UI
    display(tab)
    display(status_output)
    
    # Initialize containers
    with stream_container:
        display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">Stream Stopped</div>'))
    with captured_container:
        display(HTML('<div style="width:400px; height:300px; border:2px dashed #ccc; display:flex; align-items:center; justify-content:center;">No Image Captured</div>'))
    
    # ===== EVENT HANDLERS =====
    enable_btn.observe(on_strobe_toggle, 'value')
    period.observe(update_strobe, 'value')
    width.observe(update_strobe, 'value')
    hold_btn.observe(update_strobe, 'value')
    stream_btn.observe(toggle_stream, 'value')
    capture_btn.on_click(capture_image)

In [2]:
# Start the UI
create_ui()

Output()